<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

<br>

# <font color="#76b900">**Notebook 7:** Retrieval-Augmented Generation with Vector Stores</font>

<br>

In the previous notebook, we learned about embedding models and exercised some of their capabilities. We discussed their intended use cases of longer-form document comparison and found ways to use it as a backbone for more custom semantic comparisons. This notebook will progress these ideas toward the retrieval model's intended use case and explore how to build chatbot systems that rely on *vector stores* to automatically save and retrieve information.

<br>

### **Learning Objectives:**

- Understand how semantic-similarity-backed systems can facilitate easy-to-use retrieval formulations.

- Learn how to incorporate retrieval modules into your chat model systems for a retrieval-augmented generation (RAG) pipeline, which can be applied to tasks like document retrieval and conversation memory buffers.

<br>

### **Questions To Think About:**

- This notebook does not attempt to incorporate hierarchical reasoning or non-naive RAG (such as planning agents). Consider what modifications would be necessary to make these components work in an LCEL chain.

- Consider when it would be best to move your vector store solution into a scalable service and when a GPU will become necessary for optimization.

<br>

### **Environment Setup:**

In [1]:
# %%capture
## ^^ Comment out if you want to see the pip install process

## Necessary for Colab, not necessary for course environment
# %pip install -q langchain langchain-nvidia-ai-endpoints gradio rich
# %pip install -q arxiv pymupdf faiss-cpu

## If you encounter a typing-extensions issue, restart your runtime and try again
# from langchain_nvidia_ai_endpoints import ChatNVIDIA
# ChatNVIDIA.get_available_models()

import os
os.environ["NVIDIA_API_KEY"] = "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"

from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

In [2]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")

----

<br>

## Part 1: Summary of RAG Workflows

This notebook will explore several paradigms and derive reference code to help you approach some of the most common retrieval-augmented workflows. Specifically, the following sections will be covered (with the differences highlighted):

<br>

> ***Vector Store Workflow for Conversational Exchanges:***
- Generate semantic embedding for each new conversation.
- Add the message body to a vector store for retrieval.
- Query the vector store for relevant messages to fill in the LLM context.

<br>

> ***Modified Workflow for an Arbitrary Document:***
- **Divide the document into chunks and process them into useful messages.**
- Generate semantic embedding for each **new document chunk**.
- Add the **chunk bodies** to a vector store for retrieval.
- Query the vector store for relevant **chunks** to fill in the LLM context.
    - ***Optional:* Modify/synthesize results for better LLM results.**

<br>

> **Extended Workflow for a Directory of Arbitrary Documents:**
- Divide **each document** into chunks and process them into useful messages.
- Generate semantic embedding for each new document chunk.
- Add the chunk bodies to **a scalable vector database for fast retrieval**.
    - ***Optional*: Exploit hierarchical or metadata structures for larger systems.**
- Query the **vector database** for relevant chunks to fill in the LLM context.
    - *Optional:* Modify/synthesize results for better LLM results.

<br>

Some of the most important terminology surrounding RAG is covered in detail on the [**LlamaIndex Concepts page**](https://docs.llamaindex.ai/en/stable/getting_started/concepts.html), which itself is a great starting point for progressing towards the LlamaIndex loading and retrieving strategy. We highly recommend using it as a reference as you continue with this notebook and advise you to try out LlamaIndex after the course to consider the pros and cons firsthand!


<!-- > <img src="https://drive.google.com/uc?export=view&id=1cFbKbVvLLnFPs3yWCKIuzXkhBWh6nLQY" width=1200px/> -->
> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/data_connection_langchain.jpeg" width=1200px/>
>
> From [**Retrieval | LangChain**🦜️🔗](https://python.langchain.com/v0.1/docs/modules/data_connection/)

----

<br>

## **Part 2:** RAG for Conversation History

In our previous explorations, we delved into the capabilities of document embedding models and used them to embed, store, and compare semantic vector representations of text. Though we could motivate how to efficiently extend this into vector store land manually, the true beauty of working with a standard API is its strong incorporation with other frameworks that can already do the heavy lifting for us!

<br>

### **Step 1**: Getting A Conversation

Consider a conversation crafted using Llama-13B between a chat agent and a blue bear named Beras. This dialogue, dense with details and potential diversions, provides a rich dataset for our study:


In [3]:
conversation = [  ## This conversation was generated partially by an AI system, and modified to exhibit desirable properties
    "[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the rocky mountains?",
    "[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch across North America",
    "[Beras] Wow, that sounds amazing! Ive never been to the Rocky Mountains before, but Ive heard many great things about them.",
    "[Agent] I hope you get to visit them someday, Beras! It would be a great adventure for you!"
    "[Beras] Thank you for the suggestion! Ill definitely keep it in mind for the future.",
    "[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research online or watching documentaries about them."
    "[Beras] I live in the arctic, so I'm not used to the warm climate there. I was just curious, ya know!",
    "[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains and their significance!"
]

Using the manual embedding strategy from the previous notebook is still very viable, but we can also rest easy and let a **vector store** do all that work for us!

<br>

### **Step 2:** Constructing Our Vector Store Retriever

To streamline similarity queries on our conversation, we can employ a vector store to help keep track of passages for us! **Vector Stores**, or vector storage systems, abstract away most of the low-level details of the embedding/comparison strategies and provide a simple interface to load and compare vectors.


<!-- > <img src="https://drive.google.com/uc?export=view&id=1ZjwYbSZzsXK6ZP8O1-cY3BeRffV4oqzb" width=1000px/> -->
> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/vector_stores.jpeg" width=1200px/>
>
> From [**Vector Stores | LangChain**🦜️🔗](https://python.langchain.com/v0.1/docs/modules/data_connection/vectorstores/)

<br>

In addition to simplifying the process from an API perspective, vector stores also implement connectors, integrations, and optimizations under the hood. In our case, we will start with the [**FAISS vector store**](https://python.langchain.com/docs/integrations/vectorstores/faiss), which integrates a LangChain-compatable Embedding model with the [**FAISS (Facebook AI Similarity Search)**](https://github.com/facebookresearch/faiss) library to make the process fast and scalable on our local machine!

**Specifically:**

1. We can feed our conversation into [**a FAISS vector store**](https://python.langchain.com/docs/integrations/vectorstores/faiss) via the `from_texts` constructor. This will take our conversational data and the embedding model to create a searchable index over our discussion.
2. This vector store can then be "interpreted" as a retriever, supporting the LangChain runnable API and returning documents retrieved via an input query.

The following shows how you can construct a FAISS vector store and reinterpret it as a retriever using the LangChain `vectorstore` API:

In [4]:
%pip install faiss-cpu

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
%%time

## ^^ This cell will be timed to see how long the conversation embedding takes
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain.vectorstores import FAISS

## Streamlined from_texts FAISS vectorstore construction from text list
convstore = FAISS.from_texts(conversation, embedding=embedder)
retriever = convstore.as_retriever()

CPU times: total: 391 ms
Wall time: 1.34 s


The retriever can now be used like any other LangChain runnable to query the vector store for some relevant documents:

In [7]:
pprint(retriever.invoke("What is your name?"))

[
    Document(
        id='9883d099-03d2-4ae0-8cba-2393ef0ce2ac',
        metadata={},
        page_content="[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the 
rocky mountains?"
    ),
    Document(
        id='1a0fe0bc-edc4-44be-bbcf-c26b9061896b',
        metadata={},
        page_content='[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains
and their significance!'
    ),
    Document(
        id='25e1bfc2-fda4-4b5d-aca2-f576cc5e0c7b',
        metadata={},
        page_content='[Agent] I hope you get to visit them someday, Beras! It would be a great adventure for 
you![Beras] Thank you for the suggestion! Ill definitely keep it in mind for the future.'
    ),
    Document(
        id='64d9a851-f9ae-4a29-8f75-e298feea116e',
        metadata={},
        page_content="[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research 
online or watching documentaries about them.[Beras] I live in the arctic, so I'm not used to the warm climate 
there. I was just curious, ya know!"
    )
]

In [8]:
pprint(retriever.invoke("Where are the Rocky Mountains?"))

[
    Document(
        id='2a28586d-e559-45bf-a6ed-bb4ecc98b1f6',
        metadata={},
        page_content='[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch 
across North America'
    ),
    Document(
        id='64d9a851-f9ae-4a29-8f75-e298feea116e',
        metadata={},
        page_content="[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research 
online or watching documentaries about them.[Beras] I live in the arctic, so I'm not used to the warm climate 
there. I was just curious, ya know!"
    ),
    Document(
        id='9883d099-03d2-4ae0-8cba-2393ef0ce2ac',
        metadata={},
        page_content="[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the 
rocky mountains?"
    ),
    Document(
        id='1a0fe0bc-edc4-44be-bbcf-c26b9061896b',
        metadata={},
        page_content='[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains
and their significance!'
    )
]

As we can see, our retriever found a handful of semantically relevant documents from our query. You may notice that not all of the documents are useful or clear on their own. For example, a retrieval of *"Beras"* for *"your name"* may be problematic for the chatbot if provided out of context. Anticipating the potential problems and creating synergies between your LLM components can increase the likelihood of good RAG behavior, so keep an eye out for such pitfalls and opportunities.

<br>

### **Step 3:** Incorporating Conversation Retrieval Into Our Chain

Now that we have our loaded retriever component as a chain, we can incorporate it into our existing chat system as before. Specifically, we can start with an ***always-on RAG formulation*** where:
- **A retriever is always retrieving context by default**.
- **A generator is acting on the retrieved context**.

In [10]:
from langchain.document_transformers import LongContextReorder
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain.schema.runnable.passthrough import RunnableAssign
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from functools import partial
from operator import itemgetter

########################################################################
## Utility Runnables/Methods
def RPrint(preface=""):
    """Simple passthrough "prints, then returns" chain"""
    def print_and_return(x, preface):
        if preface: print(preface, end="")
        pprint(x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name:
            out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

## Optional; Reorders longer documents to center of output text
long_reorder = RunnableLambda(LongContextReorder().transform_documents)

In [ ]:
context_prompt = ChatPromptTemplate.from_template(
    "Answer the question using only the context"
    "\n\nRetrieved Context: {context}"
    "\n\nUser Question: {question}"
    "\nAnswer the user conversationally. User is not aware of context."
)

chain = (
    {
        'context': convstore.as_retriever() | long_reorder | docs2str,
        'question': (lambda x:x)
    }
    | context_prompt
    # | RPrint()
    | instruct_llm
    | StrOutputParser()
)

pprint(chain.invoke("Where does Beras live?"))

Based on the context provided, Beras lives in the Arctic. It seems like they're not very familiar with the warm 
climate of the Rocky Mountains.

Take a second to try out some more invocations and see how the new setup performs. Regardless of your model choice, the following questions should serve as interesting starting points.

In [ ]:
pprint(chain.invoke("Where are the Rocky Mountains?"))

Hello there! The Rocky Mountains are a stunning range of mountains that span across North America. Unfortunately, 
without more context, I can't give you a precise location beyond that. However, if you're interested, you can learn
more about them by doing some online research or watching informative documentaries!

In [ ]:
pprint(chain.invoke("Do you know Rocky Mountains? If so, tell me the location now"))

Absolutely, I can help with that! The Rocky Mountains, often referred to as the "Rockies," are a major mountain 
range in western North America. They stretch all the way from the northernmost part of British Columbia, in western
Canada, to New Mexico in the Southwestern United States. So, they span across several states and provinces in North
America.

In [ ]:
pprint(chain.invoke("Where are the Rocky Mountains? Are they close to California?"))

The Rocky Mountains are a stunning mountain range that extends across North America. While they're not directly 
close to California, they do reach as far south as New Mexico. If you're traveling from California, you could 
certainly visit some parts of the Rocky Mountains with a bit of journeying. They're truly a sight to behold!

In [ ]:
pprint(chain.invoke("How far away is Beras from the Rocky Mountains?"))

Unfortunately, the context doesn't provide information on Beras' exact location or distance from the Rocky 
Mountains. I do know that Beras mentioned living in the Arctic, which is quite far from the Rockies geographically.
The Rocky Mountains are a vast mountain range located mostly in the western United States and Canada.

<br>

You might notice some decent performance with this always-on retrieval node in the loop since the actual context being fed into the LLM remains relatively small. It's important to experiment with factors like embedding sizes, context limits, and model options to see what kinds of behavior you can expect and which efforts are worth taking to improve performance.

<br>

### **Step 4:** Automatic Conversation Storage

Now that we see how our vector store memory unit should function, we can perform one last integration to allow our conversation to add new entries to our conversation: a runnable that calls the `add_texts` method for us to update the store state.


In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

########################################################################
## Reset knowledge base and define what it means to add more messages.
convstore = FAISS.from_texts(conversation, embedding=embedder)

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([f"User said {d.get('input')}", f"Agent said {d.get('output')}"])
    return d.get('output')

########################################################################

# instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")

chat_prompt = ChatPromptTemplate.from_template(
    "Answer the question using only the context"
    "\n\nRetrieved Context: {context}"
    "\n\nUser Question: {input}"
    "\nAnswer the user conversationally. Make sure the conversation flows naturally.\n"
    "[Agent]"
)


conv_chain = (
    {
        'context': convstore.as_retriever() | long_reorder | docs2str,
        'input': (lambda x:x)
    }
    | RunnableAssign({'output' : chat_prompt | instruct_llm | StrOutputParser()})
    | partial(save_memory_and_get_output, vstore=convstore)
)

pprint(conv_chain.invoke("I'm glad you agree! I can't wait to get some ice cream there! It's such a good food!"))
print()
pprint(conv_chain.invoke("Can you guess what my favorite food is?"))
print()
pprint(conv_chain.invoke("Actually, my favorite is honey! Not sure where you got that idea?"))
print()
pprint(conv_chain.invoke("I see! Fair enough! Do you know my favorite food now?"))

While I'm sure the Rocky Mountains would provide a magnificent backdrop for enjoying ice cream, it seems like your 
excitement has taken a tasty turn! The Rockies are indeed awe-inspiring, spanning North America and offering 
breathtaking views and adventures. However, let's not forget about that delicious ice cream! While it may not be a 
traditional part of the Rockies experience, there's no reason we can't incorporate it into our daydreams about 
visiting this stunning mountain range! It's always great to keep our taste buds in mind while planning our 
explorations.

Given our conversation, I'd wager a guess that ice cream might just be your favorite food! After all, you did 
express your enthusiasm about enjoying some ice cream amidst the stunning views of the Rocky Mountains. Remember, 
though, this is just a friendly guess based on our recent chat.

I see! Well, I must admit, the joy you shared about savoring ice cream while surrounded by the awe-inspiring views 
of the Rocky Mountains led me to believe it was your favorite. Ice cream is indeed delightful, but honey is just as
wonderful! Just like how the Rockies can't be fully appreciated without seeing them firsthand, the true essence of 
honey is best experienced by tasting it. It's always exciting to discover more about one's preferences, isn't it? 
The world is full of sweet surprises, and honey is definitely one of them.

Of course! Based on our conversation, I now know that honey is your favorite food! It turns out, my guess about ice
cream wasn't quite on the mark, but it led to a sweet discovery nonetheless. Now, I'm even more curious to hear 
about other foods you cherish. Is there any specific reason why honey holds a special place in your heart?

Unlike the more automatic full-text or rule-based approaches to injecting context into the LLM, this approach ensures some amount of consolidation which can keep the context length from getting out of hand. It's still not a full-proof strategy on its own, but it's a stark improvement for unstructured conversations (and doesn't even require a strong instruction-tuned model to perform slot-filling).

----

<br>

## **Part 3 [Exercise]:** RAG For Document Chunk Retrieval

Given our prior exploration of document loading, the idea that data chunks can be embedded and searched through probably isn't surprising. With that said, it is definitely worth going over since applying RAG with documents is a double-edged sword; it may **seem** to work well out of the box but requires some extra care when optimizing it for truly reliable performance. It also provides an excellent opportunity to review some fundamental LCEL skills, so let's see what we can do!

<br>

### **Exercise:**

In the previous example, you may recall that we pulled in some relatively small papers with the help of [`ArxivLoader`](https://python.langchain.com/docs/integrations/document_loaders/arxiv) using the following syntax:

```python
from langchain.document_loaders import ArxivLoader

docs = [
    ArxivLoader(query="2205.00445").load(),  ## MRKL
    ArxivLoader(query="2210.03629").load(),  ## ReAct
]
```

Given all that you've learned so far, choose a selection of papers that you would like to use and develop a chatbot that can talk about them!

<br>

Though this is a pretty big task, a walkthrough of ***most*** of the process will be provided below. By the end of the walkthrough, many of the necessary puzzle pieces will be provided, and your real task will be to integrate them together for the final `retrieval_chain`. When you're done, get ready to re-integrate the chain (or a flavor of your choice) in the last notebook as part of the evaluation exercise!


<br>

### **Task 1**: Loading And Chunking Your Documents

The following code block gives you some default papers to load in for your RAG chain. Feel free to select more papers as desired, but note that longer documents will take longer to process. A few simplifying assumptions and additional processing steps are included to help you improve your naive RAG performance:

- Documents are cut off prior to the "References" section if one exists. This will keep our system from considering the citations and appendix sections, which tend to be long and distracting.

- A chunk that lists the available documents is inserted to provide a high-level view of all available documents in a single chunk. If your pipeline does not provide metadata on each retrieval, this is a useful component and can even be listed among a list of higher-priority pieces if appropriate.

- Additionally, the metadata entries are also inserted to provide general information. Ideally, there would also be some synthetic chunks that merge the metadata into interesting cross-document chunks.

**NOTE:** ***For the sake of the assessment, please include at least one paper that is less than one month old!***


In [12]:
import json
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import ArxivLoader

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100,
    separators=["\n\n", "\n", ".", ";", ",", " "],
)

## TODO: Please pick some papers and add them to the list as you'd like
## NOTE: To re-use for the final assessment, make sure at least one paper is < 1 month old
print("Loading Documents")
docs = [
    ArxivLoader(query="1706.03762").load(),  ## Attention Is All You Need Paper
    ArxivLoader(query="1810.04805").load(),  ## BERT Paper
    ArxivLoader(query="2005.11401").load(),  ## RAG Paper
    ArxivLoader(query="2205.00445").load(),  ## MRKL Paper
    ArxivLoader(query="2310.06825").load(),  ## Mistral Paper
    ArxivLoader(query="2306.05685").load(),  ## LLM-as-a-Judge
    ArxivLoader(query="2511.03948").load(),  ## Extracting Causal Relations in Deep Knowledge Tracing

    ## Some longer papers
    # ArxivLoader(query="2210.03629").load(),  ## ReAct Paper
    # ArxivLoader(query="2112.10752").load(),  ## Latent Stable Diffusion Paper
    # ArxivLoader(query="2103.00020").load(),  ## CLIP Paper
    ## TODO: Feel free to add more
]

## Cut the paper short if references is included.
## This is a standard string in papers.
for doc in docs:
    content = json.dumps(doc[0].page_content)
    if "References" in content:
        doc[0].page_content = content[:content.index("References")]

## Split the documents and also filter out stubs (overly short chunks)
print("Chunking Documents")
docs_chunks = [text_splitter.split_documents(doc) for doc in docs]
docs_chunks = [[c for c in dchunks if len(c.page_content) > 200] for dchunks in docs_chunks]

## Make some custom Chunks to give big-picture details
doc_string = "Available Documents:"
doc_metadata = []
for chunks in docs_chunks:
    metadata = getattr(chunks[0], 'metadata', {})
    doc_string += "\n - " + metadata.get('Title')
    doc_metadata += [str(metadata)]

extra_chunks = [doc_string] + doc_metadata

## Printing out some summary information for reference
pprint(doc_string, '\n')
for i, chunks in enumerate(docs_chunks):
    print(f"Document {i}")
    print(f" - # Chunks: {len(chunks)}")
    print(f" - Metadata: ")
    pprint(chunks[0].metadata)
    print()

Loading Documents
Chunking Documents


Available Documents:
 - Attention Is All You Need
 - BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding
 - Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks
 - MRKL Systems: A modular, neuro-symbolic architecture that combines large language models, external knowledge 
sources and discrete reasoning
 - Mistral 7B
 - Judging LLM-as-a-Judge with MT-Bench and Chatbot Arena
 - Extracting Causal Relations in Deep Knowledge Tracing 

Document 0
 - # Chunks: 35
 - Metadata: 


{
    'Published': '2023-08-02',
    'Title': 'Attention Is All You Need',
    'Authors': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz 
Kaiser, Illia Polosukhin',
    'Summary': 'The dominant sequence transduction models are based on complex recurrent or convolutional neural 
networks in an encoder-decoder configuration. The best performing models also connect the encoder and decoder 
through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on 
attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation 
tasks show these models to be superior in quality while being more parallelizable and requiring significantly less 
time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the 
existing best results, including ensembles by over 2 BLEU. On the WMT 2014 English-to-French translation task, our 
model establishes a new single-model state-of-the-art BLEU score of 41.8 after training for 3.5 days on eight GPUs,
a small fraction of the training costs of the best models from the literature. We show that the Transformer 
generalizes well to other tasks by applying it successfully to English constituency parsing both with large and 
limited training data.'
}


Document 1
 - # Chunks: 45
 - Metadata: 


...

{
    'Published': '2023-10-10',
    'Title': 'Mistral 7B',
    'Authors': 'Albert Q. Jiang, Alexandre Sablayrolles, Arthur Mensch, Chris Bamford, Devendra Singh Chaplot, 
Diego de las Casas, Florian Bressand, Gianna Lengyel, Guillaume Lample, Lucile Saulnier, Lélio Renard Lavaud, 
Marie-Anne Lachaux, Pierre Stock, Teven Le Scao, Thibaut Lavril, Thomas Wang, Timothée Lacroix, William El Sayed',
    'Summary': 'We introduce Mistral 7B v0.1, a 7-billion-parameter language model engineered for superior 
performance and efficiency. Mistral 7B outperforms Llama 2 13B across all evaluated benchmarks, and Llama 1 34B in 
reasoning, mathematics, and code generation. Our model leverages grouped-query attention (GQA) for faster 
inference, coupled with sliding window attention (SWA) to effectively handle sequences of arbitrary length with a 
reduced inference cost. We also provide a model fine-tuned to follow instructions, Mistral 7B -- Instruct, that 
surpasses the Llama 2 13B -- Chat model both on human and automated benchmarks. Our models are released under the 
Apache 2.0 license.'
}


Document 5
 - # Chunks: 44
 - Metadata: 


{
    'Published': '2023-12-24',
    'Title': 'Judging LLM-as-a-Judge with MT-Bench and Chatbot Arena',
    'Authors': 'Lianmin Zheng, Wei-Lin Chiang, Ying Sheng, Siyuan Zhuang, Zhanghao Wu, Yonghao Zhuang, Zi Lin, 
Zhuohan Li, Dacheng Li, Eric P. Xing, Hao Zhang, Joseph E. Gonzalez, Ion Stoica',
    'Summary': 'Evaluating large language model (LLM) based chat assistants is challenging due to their broad 
capabilities and the inadequacy of existing benchmarks in measuring human preferences. To address this, we explore 
using strong LLMs as judges to evaluate these models on more open-ended questions. We examine the usage and 
limitations of LLM-as-a-judge, including position, verbosity, and self-enhancement biases, as well as limited 
reasoning ability, and propose solutions to mitigate some of them. We then verify the agreement between LLM judges 
and human preferences by introducing two benchmarks: MT-bench, a multi-turn question set; and Chatbot Arena, a 
crowdsourced battle platform. Our results reveal that strong LLM judges like GPT-4 can match both controlled and 
crowdsourced human preferences well, achieving over 80% agreement, the same level of agreement between humans. 
Hence, LLM-as-a-judge is a scalable and explainable way to approximate human preferences, which are otherwise very 
expensive to obtain. Additionally, we show our benchmark and traditional benchmarks complement each other by 
evaluating several variants of LLaMA and Vicuna. The MT-bench questions, 3K expert votes, and 30K conversations 
with human preferences are publicly available at https://github.com/lm-sys/FastChat/tree/main/fastchat/llm_judge.'
}


Document 6
 - # Chunks: 30
 - Metadata: 


{
    'Published': '2025-11-06',
    'Title': 'Extracting Causal Relations in Deep Knowledge Tracing',
    'Authors': 'Kevin Hong, Kia Karbasi, Gregory Pottie',
    'Summary': "A longstanding goal in computational educational research is to develop explainable knowledge 
tracing (KT) models. Deep Knowledge Tracing (DKT), which leverages a Recurrent Neural Network (RNN) to predict 
student knowledge and performance on exercises, has been proposed as a major advancement over traditional KT 
methods. Several studies suggest that its performance gains stem from its ability to model bidirectional 
relationships between different knowledge components (KCs) within a course, enabling the inference of a student's 
understanding of one KC from their performance on others. In this paper, we challenge this prevailing explanation 
and demonstrate that DKT's strength lies in its implicit ability to model prerequisite relationships as a causal 
structure, rather than bidirectional relationships. By pruning exercise relation graphs into Directed Acyclic 
Graphs (DAGs) and training DKT on causal subsets of the Assistments dataset, we show that DKT's predictive 
capabilities align strongly with these causal structures. Furthermore, we propose an alternative method for 
extracting exercise relation DAGs using DKT's learned representations and provide empirical evidence supporting our
claim. Our findings suggest that DKT's effectiveness is largely driven by its capacity to approximate causal 
dependencies between KCs rather than simple relational mappings."
}

<br>

### **Task 2**: Construct Your Document Vector Stores

Now that we have all of the components, we can go ahead and create indices surrounding them:

In [13]:
%%time
print("Constructing Vector Stores")
vecstores = [FAISS.from_texts(extra_chunks, embedder)]
vecstores += [FAISS.from_documents(doc_chunks, embedder) for doc_chunks in docs_chunks]

Constructing Vector Stores
CPU times: total: 2.97 s
Wall time: 21.2 s


<br>

From there, we can combine our indices into a single one using the following utility:

In [14]:
from faiss import IndexFlatL2
from langchain_community.docstore.in_memory import InMemoryDocstore

embed_dims = len(embedder.embed_query("test"))
def default_FAISS():
    '''Useful utility for making an empty FAISS vectorstore'''
    return FAISS(
        embedding_function=embedder,
        index=IndexFlatL2(embed_dims),
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
        normalize_L2=False
    )

def aggregate_vstores(vectorstores):
    ## Initialize an empty FAISS Index and merge others into it
    ## We'll use default_faiss for simplicity, though it's tied to your embedder by reference
    agg_vstore = default_FAISS()
    for vstore in vectorstores:
        agg_vstore.merge_from(vstore)
    return agg_vstore

## Unintuitive optimization; merge_from seems to optimize constituent vector stores away
docstore = aggregate_vstores(vecstores)

print(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")

Constructed aggregate docstore with 269 chunks


<br>

### **Task 3: [Exercise]** Implement Your RAG Chain

Finally, all the puzzle pieces are in place to implement the RAG pipeline! As a review, we now have:

- A way to construct a from-scratch vector store for conversational memory (and a way to initialize an empty one with `default_FAISS()`)

- A vector store pre-loaded with useful document information from our `ArxivLoader` utility (stored in `docstore`).

With the help of a couple more utilities, you're finally ready to integrate your chain! A few additional convenience utilities are provided (`doc2str` and the now-common `RPrint`) but are optional to use. Additionally, some starter prompts and structures are also defined.

> **Given all of this:** Please implement the `retrieval_chain`.

In [20]:
from langchain.document_transformers import LongContextReorder
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

import gradio as gr
from functools import partial
from operator import itemgetter

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")
# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1")
# instruct_llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")

convstore = default_FAISS()

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([
        f"User previously responded with {d.get('input')}",
        f"Agent previously responded with {d.get('output')}"
    ])
    return d.get('output')

initial_msg = (
    "Hello! I am a document chat agent here to help the user!"
    f" I have access to the following documents: {doc_string}\n\nHow can I help you?"
)

chat_prompt = ChatPromptTemplate.from_messages([("system",
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked: {input}\n\n"
    " From this, we have retrieved the following potentially-useful info: "
    " Conversation History Retrieval:\n{history}\n\n"
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational.)"
), ('user', '{input}')])

stream_chain = chat_prompt| RPrint() | instruct_llm | StrOutputParser()

################################################################################################
## BEGIN TODO: Implement the retrieval chain to make your system work!

retrieval_chain = (
    {'input' : (lambda x: x)}
    ## TODO: Make sure to retrieve history & context from convstore & docstore, respectively.
    ## HINT: Our solution uses RunnableAssign, itemgetter, long_reorder, and docs2str
    | RunnableAssign({'history' : itemgetter('input') | convstore.as_retriever() | long_reorder | docs2str})
    | RunnableAssign({'context' : itemgetter('input') | docstore.as_retriever() | long_reorder | docs2str})
)

## END TODO
################################################################################################

def chat_gen(message, history=[], return_buffer=True):
    buffer = ""
    ## First perform the retrieval based on the input message
    retrieval = retrieval_chain.invoke(message)
    line_buffer = ""

    ## Then, stream the results of the stream_chain
    for token in stream_chain.stream(retrieval):
        buffer += token
        ## If you're using standard print, keep line from getting too long
        yield buffer if return_buffer else token

    ## Lastly, save the chat exchange to the conversation memory buffer
    save_memory_and_get_output({'input':  message, 'output': buffer}, convstore)


## Start of Agent Event Loop
test_question = "DKT is what!"  ## <- modify as desired

## Before you launch your gradio interface, make sure your thing works
for response in chat_gen(test_question, return_buffer=False):
    print(response, end='')

ChatPromptValue(
    messages=[
        SystemMessage(
            content='You are a document chatbot. Help the user as they ask questions about documents. User messaged
just asked: DKT is what!\n\n From this, we have retrieved the following potentially-useful info:  Conversation 
History Retrieval:\n\n\n Document Retrieval:\n[Quote from Extracting Causal Relations in Deep Knowledge Tracing] 
ercise relation discovery [15]. Their method assigns an in-\nfluence score Jij to every directed pair of exercises 
i and\nj, which is the conditional probability of correctly answer-\ning exercise j after correctly answering 
exercise i in the\nprevious timestep, normalized by the sum of such condi-\ntional probabilities. The influence 
score from i to j quan-\narXiv:2511.03948v1  [cs.AI]  6 Nov 2025\ntifies the prerequisite dependency of the concept
i on the\nlearning concept j. We refer to this method as the DKT\nmethod.\nDKT’s superior performance can be 
attributed\nto the model’s ability to learn these influence-based depen-\ndencies to estimate student knowledge. 
Many studies have\nincorporated exercise relations into subsequent KT mod-\nels to improve performance and enhance 
interpretability in\nthe prediction process.\nFor example, Hierarchical Graph\nKnowledge Tracing (HGKT) [17], 
Structure-based Knowl-\nedge Tracing (SKT) [18], and Deep Knowledge Tracing with\n[Quote from Extracting Causal 
Relations in Deep Knowledge Tracing] structure. We formalize this by pruning exercise relations\ngraphs into 
Directed Acyclic Graphs (DAGs) to reflect pre-\nrequisite relations, and show that DKT’s predictive perfor-\nmance 
improves when trained on data filtered through these\ncausal structures. We also introduce an alternative 
method\nfor extracting exercise relations that yields accurate and\nmore stable representations of student 
knowledge and un-\nderlying concept dependencies. To facilitate future research\non these ideas, we have published 
our code 1.\n2.\nMETHODOLOGY\nWe conduct our study using the Assistments datasets, which\nare among the largest 
publicly available KT datasets and are\nwidely used as benchmarks for KT models [1, 8, 16]. These\ndatasets capture
student interactions over extended periods\nof time and across a wide range of exercises in grade 
school\nmathematics. We utilize three datasets: Assistments 2009\n2 (skill builder data corrected collapsed), 
Assistments 2012 3\n[Quote from Extracting Causal Relations in Deep Knowledge Tracing] gent tutoring systems [9, 
11], enables personalizing activities\nto suit individuals with varying levels of proficiency. A key\ncomponent of 
these adaptive systems is knowledge tracing\n(KT), which aims to estimate students’ mastery of several\nexercise 
concepts, known as knowledge components (KCs),\nas they interact with the corresponding exercises [2, 5]. 
For-\nmally, xi = {et, at} represents a student’s answer pair, where\net represents the exercise ID, and at 
represents whether the\nstudent answered correctly or incorrectly. Given a series of\npast interactions, X = {x1, 
x2, . . . , xt} and the next con-\ncept exercise, et+1, the task of the KT model is to estimate\nthe likelihood of 
the student answering correctly, at+1 [6].\nTraditionally, Bayesian Knowledge Tracing (BKT) [6] was\nused to 
perform KT. Despite its popularity, BKT is often\ncritiqued for its binary representation of knowledge 
states\n(mastered or not mastered) which oversimplifies the learn-\n[Quote from Extracting Causal Relations in Deep
Knowledge Tracing] (mastered or not mastered) which oversimplifies the learn-\ning process. Several models were 
later proposed to address\nthese limitations. Learning Factors Analysis\n[4] improved\nupon BKT by representing 
learning as a continuous pro-\ncess influenced by multiple exercise interactions and accu-\nmulated practice. 
Performance Factors Analysis [10] further\ncaptured the complexity of learning by tracking the effects\nof correct 
and incorrect

DKT, or Deep Knowledge Tracing, is a method used in the field of education to estimate student knowledge and learning progression. It's a data-driven approach that uses machine learning algorithms to model students' mastery of various knowledge components (KCs) as they interact with corresponding exercises. DKT is known for its ability to implicitly infer relationships between exercises, which can be prerequisite dependencies or corequisites. This method outperforms traditional techniques like Bayesian Knowledge Tracing (BKT) by representing learning as a continuous process influenced by multiple exercise interactions.

The DKT method assigns an influence score Jij to every directed pair of exercises i and j, which is the conditional probability of correctly answering exercise j after correctly answering exercise i in the previous timestep. This influence score quantifies the prerequisite dependency of the concept i on the learning concept j. The DKT's superior performance is attribute

### **Task 4:** Interact With Your Gradio Chatbot

In [21]:
chatbot = gr.Chatbot(value = [[None, initial_msg]])
demo = gr.ChatInterface(chat_gen, chatbot=chatbot).queue()

try:
    demo.launch(debug=True, share=True, show_api=False)
    demo.close()
except Exception as e:
    demo.close()
    print(e)
    raise e

C:\Users\okafo\AppData\Local\Temp\ipykernel_8348\1029931177.py:1: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(value = [[None, initial_msg]])
C:\Users\okafo\AppData\Roaming\Python\Python312\site-packages\gradio\chat_interface.py:331: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://41821a80a985d60eeb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ChatPromptValue(
    messages=[
        SystemMessage(
            content='You are a document chatbot. Help the user as they ask questions about documents. User messaged
just asked: I am interested in DKT\n\n From this, we have retrieved the following potentially-useful info:  
Conversation History Retrieval:\n[Quote from Document] User previously responded with DKT is what!\n[Quote from 
Document] Agent previously responded with DKT, or Deep Knowledge Tracing, is a method used in the field of 
education to estimate student knowledge and learning progression. It\'s a data-driven approach that uses machine 
learning algorithms to model students\' mastery of various knowledge components (KCs) as they interact with 
corresponding exercises. DKT is known for its ability to implicitly infer relationships between exercises, which 
can be prerequisite dependencies or corequisites. This method outperforms traditional techniques like Bayesian 
Knowledge Tracing (BKT) by representing learning as a continuous process influenced by multiple exercise 
interactions.\n\nThe DKT method assigns an influence score Jij to every directed pair of exercises i and j, which 
is the conditional probability of correctly answering exercise j after correctly answering exercise i in the 
previous timestep. This influence score quantifies the prerequisite dependency of the concept i on the learning 
concept j. The DKT\'s superior performance is attributed to the model\'s ability to learn these influence-based 
dependencies.\n\nTo clarify further, the DKT method is detailed in the paper "Extracting Causal Relations in Deep 
Knowledge Tracing" (arXiv:2511.03948v1 [cs.AI], November 6, 2025). The authors discuss how DKT improves predictive 
performance when trained on data filtered through Directed Acyclic Graphs (DAGs) that reflect prerequisite 
relations. Moreover, DKT has been applied in models like Hierarchical Graph Knowledge Tracing (HGKT), 
Structure-based Knowledge Tracing (SKT), and Deep Knowledge Tracing with Structure, highlighting the significance 
of incorporating exercise relations into subsequent KT models for better interpretability in the prediction 
process.\n\n\n Document Retrieval:\n[Quote from Extracting Causal Relations in Deep Knowledge Tracing] (mastered or
not mastered) which oversimplifies the learn-\ning process. Several models were later proposed to address\nthese 
limitations. Learning Factors Analysis\n[4] improved\nupon BKT by representing learning as a continuous pro-\ncess 
influenced by multiple exercise interactions and accu-\nmulated practice. Performance Factors Analysis [10] 
further\ncaptured the complexity of learning by tracking the effects\nof correct and incorrect prior exercise 
attempts on perfor-\nmance. These developments laid the groundwork for Deep\nKnowledge Tracing (DKT) [15], which 
popularized a key\nextension to KT: the ability to implicitly infer relationships\nbetween exercises.\nThe 
relationships between exercises can take the form of\nprerequisite dependencies, where understanding one 
exer-\ncise improves performance on another, or corequisites, where\nexercises depend on each other. Researchers 
have demon-\nstrated that exercise relationships can be mapped into an ex-\n[Quote from Extracting Causal Relations
in Deep Knowledge Tracing] 2 (skill builder data corrected collapsed), Assistments 2012 3\n(2012-2013 data with 
predictions 4 final), and Assistments\n2017 4 (anonymized full release competition dataset). The\n2009 dataset 
contains 346,860 exercise attempts from 4,217\nstudents across 123 exercises.\nThe 2012 dataset includes\n6,123,270
attempts from 46,674 students across 265 exer-\ncises. The 2017 dataset consists of 942,816 attempts from\n1,709 
students across 102 exercises.\nThe datasets do not\ncontain any personal information.\nWe use DKT, implemented via
the pyKT library [13], 
to\n1https://github.com/kevinhongca/\ndkt-causal-relations\n2https://sites.google.com/site/assistmentsdata/\

ChatPromptValue(
    messages=[
        SystemMessage(
            content='You are a document chatbot. Help the user as they ask questions about documents. User messaged
just asked: Okay thank you. Do you know finding nemo\n\n From this, we have retrieved the following 
potentially-useful info:  Conversation History Retrieval:\n[Quote from Document] Agent previously responded with 
Deep Knowledge Tracing, or DKT, is a method used in the field of education to estimate student knowledge and 
learning progression. It\'s a data-driven approach that uses machine learning algorithms to model students\' 
mastery of various knowledge components as they interact with corresponding exercises (Extracting Causal Relations 
in Deep Knowledge Tracing, arXiv:2511.03948v1 [cs.AI], November 6, 2025).\n\nDKT is known for its ability to 
implicitly infer relationships between exercises, which can be prerequisite dependencies or corequisites. This 
method outperforms traditional techniques like Bayesian Knowledge Tracing (BKT) by representing learning as a 
continuous process influenced by multiple exercise interactions.\n\nThe DKT method assigns an influence score Jij 
to every directed pair of exercises i and j, which is the conditional probability of correctly answering exercise j
after correctly answering exercise i in the previous timestep. This influence score quantifies the prerequisite 
dependency of the concept i on the learning concept j (Extracting Causal Relations in Deep Knowledge Tracing, 
arXiv:2511.03948v1 [cs.AI], November 6, 2025).\n\nDKT has been applied in models like Hierarchical Graph Knowledge 
Tracing (HGKT), Structure-based Knowledge Tracing (SKT), and Deep Knowledge Tracing with Structure, highlighting 
the significance of incorporating exercise relations into subsequent KT models for better interpretability in the 
prediction process (Extracting Causal Relations in Deep Knowledge Tracing, arXiv:2511.03948v1 [cs.AI], November 6, 
2025).\n\nIn the paper you mentioned, DKT was trained on data filtered through Directed Acyclic Graphs (DAGs) that 
reflect prerequisite relations, improving its predictive performance. The datasets used to train and test the DKT 
model include Assistments 2009, 2012, and 2017, all of which are publicly available and do not contain any personal
information (Extracting Causal Relations in Deep Knowledge Tracing, arXiv:2511.03948v1 [cs.AI], November 6, 
2025).\n[Quote from Document] User previously responded with I am interested in DKT\n[Quote from Document] User 
previously responded with DKT is what!\n[Quote from Document] Agent previously responded with DKT, or Deep 
Knowledge Tracing, is a method used in the field of education to estimate student knowledge and learning 
progression. It\'s a data-driven approach that uses machine learning algorithms to model students\' mastery of 
various knowledge components (KCs) as they interact with corresponding exercises. DKT is known for its ability to 
implicitly infer relationships between exercises, which can be prerequisite dependencies or corequisites. This 
method outperforms traditional techniques like Bayesian Knowledge Tracing (BKT) by representing learning as a 
continuous process influenced by multiple exercise interactions.\n\nThe DKT method assigns an influence score Jij 
to every directed pair of exercises i and j, which is the conditional probability of correctly answering exercise j
after correctly answering exercise i in the previous timestep. This influence score quantifies the prerequisite 
dependency of the concept i on the learning concept j. The DKT\'s superior performance is attributed to the 
model\'s ability to learn these influence-based dependencies.\n\nTo clarify further, the DKT method is detailed in 
the paper "Extracting Causal Relations in Deep Knowledge Tracing" (arXiv:2511.03948v1 [cs.AI], November 6, 2025). 
The authors discuss how DKT improves predictive performance when trained on data filtered through Directed Acycl

ChatPromptValue(
    messages=[
        SystemMessage(
            content='You are a document chatbot. Help the user as they ask questions about documents. User messaged
just asked: Haha, it\'s not common knowledge. They are people who haven\'t watched or heard of it\n\n From this, we
have retrieved the following potentially-useful info:  Conversation History Retrieval:\n[Quote from Document] User 
previously responded with Okay thank you. Do you know finding nemo\n[Quote from Document] User previously responded
with I am interested in DKT\n[Quote from Document] Agent previously responded with It seems like you\'ve asked 
about "Finding Nemo" all of a sudden, without any prior context or relation to the Deep Knowledge Tracing (DKT) 
topic we were discussing. I assume you\'d like to know more about the movie, "Finding Nemo."\n\n"Finding Nemo" is a
popular 2003 computer-animated adventure film produced by Pixar Animation Studios and released by Walt Disney 
Pictures. The film follows the journey of Marlin, a clownfish, who searches for his abducted son, Nemo, across the 
vast ocean. Throughout their adventure, they meet various interesting sea creatures and learn valuable lessons 
about courage, family, and friendship. The movie was a massive critical and commercial success and has since become
a modern classic in animation.\n\nSources:\n[Quote from Document] None, as the information provided is common 
knowledge about the movie "Finding Nemo."\n[Quote from Document] User previously responded with DKT is what!\n\n\n 
Document Retrieval:\n[Quote from MRKL Systems: A modular, neuro-symbolic architecture that combines large language 
models, external knowledge sources and discrete reasoning] . While\\nLMs indeed model syntax, and other linguistic 
elements, their most striking feature\\nis that they model the world, as described by the data on which they were 
trained.\\n1\\narXiv:2205.00445v1  [cs.CL]  1 May 2022\\nAnd so really LMs serve as a textual gateway to the 
universe of knowledge [11, 12],\\nand perhaps should instead be called \\u201clanguage and knowledge\\u201d 
models.\\nWhen viewed this way, it becomes clear that, despite their value, current LMs\\nhave inherent 
limitations. While versatile and impressive, the output of even huge\\nLMs is in many cases wrong, and often 
ridiculously so [13]. Here is a sample output\\nof GPT-3 on some simple queries. (To be clear, this is not a 
critique of GPT-3 specif-\\nically, and other LMs \\u2014 including our own Jurassic-1 \\u2014 exhibit similar 
silliness\n[Quote from Extracting Causal Relations in Deep Knowledge Tracing] W. Luo. pyKT: A Python Library to 
Benchmark Deep\nLearning based Knowledge Tracing Models, Jan. 2023.\narXiv:2206.11460 [cs].\n[14] H. Nakagawa, Y. 
Iwasawa, and Y. Matsuo.\nGraph-based Knowledge Tracing: Modeling Student\nProficiency Using Graph Neural Network. 
In\nIEEE/WIC/ACM International Conference on Web\nIntelligence, pages 156–163, Thessaloniki Greece, Oct.\n2019. 
ACM.\n[15] C. Piech, J. Spencer, J. Huang, S. Ganguli,\nM. Sahami, L. Guibas, and J. Sohl-Dickstein. 
Deep\nKnowledge Tracing, June 2015. arXiv:1506.05908 [cs].\n[16] E. Prihar, Manaal Syed, Korinn Ostrow, S. 
Shaw,\nA. Sales, and N. Heffernan. Exploring Common\nTrends in Online Educational Experiments. July 
2022.\nPublisher: Zenodo.\n[17] H. Tong, Z. Wang, Y. Zhou, S. Tong, W. Han, and\nQ. Liu. HGKT: Introducing 
Hierarchical Exercise\nGraph for Knowledge Tracing, Aug. 2022.\narXiv:2006.16915 [cs].\n[18] S. Tong, Q. Liu, W. 
Huang, Z. Hunag, E. Chen,\nC. Liu, H. Ma, and S. Wang. Structure-Based\n[Quote from Retrieval-Augmented Generation 
for Knowledge-Intensive NLP Tasks] "Retrieval-Augmented Generation for\\nKnowledge-Intensive NLP Tasks\\nPatrick 
Lewis\\u2020\\u2021, Ethan Perez\\u22c6,\\nAleksandra Piktus\\u2020, Fabio Petroni\\u2020, Vladimir 
Karpukhin\\u2020, Naman Goyal\\u2020, Heinrich K\\u00fcttler\\u2020,\\nMike Lewis\\u2020, Wen-tau Yih\\u2020, Tim 
Rockt\\u00e4schel\\u

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://41821a80a985d60eeb.gradio.live
Closing server running on port: 7860


<br>

----

<br>

## **Part 4:** Saving Your Index For Evaluation

After you've implemented your RAG chain, please save your accumulated vector store as shown [in the official documentation](https://python.langchain.com/docs/integrations/vectorstores/faiss#saving-and-loading). You'll have a chance to use it again for your final assessment!

In [22]:
## Save and compress your index
docstore.save_local("docstore_index")
!tar czvf docstore_index.tgz docstore_index

!rm -rf docstore_index

a docstore_index
a docstore_index/index.faiss
a docstore_index/index.pkl
'rm' is not recognized as an internal or external command,
operable program or batch file.


If everything was properly saved, the following line can be invoked to pull the index from the compressed `tgz` file (assuming the pip requirements are installed). After you have confirmed that the cell can pull in your index, download `docstore_index.tgz` for use in the last notebook!

In [24]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS

# embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")
!tar xzvf docstore_index.tgz
new_db = FAISS.load_local("docstore_index", embedder, allow_dangerous_deserialization=True)
docs = new_db.similarity_search("DKT")
print(docs[0].page_content[:1000])

x docstore_index/
x docstore_index/index.faiss
x docstore_index/index.pkl


(mastered or not mastered) which oversimplifies the learn-
ing process. Several models were later proposed to address
these limitations. Learning Factors Analysis
[4] improved
upon BKT by representing learning as a continuous pro-
cess influenced by multiple exercise interactions and accu-
mulated practice. Performance Factors Analysis [10] further
captured the complexity of learning by tracking the effects
of correct and incorrect prior exercise attempts on perfor-
mance. These developments laid the groundwork for Deep
Knowledge Tracing (DKT) [15], which popularized a key
extension to KT: the ability to implicitly infer relationships
between exercises.
The relationships between exercises can take the form of
prerequisite dependencies, where understanding one exer-
cise improves performance on another, or corequisites, where
exercises depend on each other. Researchers have demon-
strated that exercise relationships can be mapped into an ex-


-----

<br>

## **Part 5:** Wrap-Up

Congratulations! Assuming your RAG chain is all good, you're now ready to move on to the **RAG Evaluation [Assessment]** section!

### <font color="#76b900">**Great Job!**</font>

### **Next Steps:**
1. **[Optional]** Revisit the **"Questions To Think About" Section** at the top of the notebook and think about some possible answers.

---

<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>